In [1]:
import os
import sys
import pandas as pd
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname("ETL"))))

In [2]:
from ETL import T


함수에 대한 설명은 아래와 같습니다. 

라이브러리는 ETL과정중 T(Transform)을 포함하고 있습니다. 

데이터베이스에서 데이터를 확인하는 클래스는 Checker 입니다. 

클래스 지정시 데이터베이스(postgresql)의 id, pw, ip, db, 추출하고자 하는 테이블명을 입력해주세요. 

read_excel() 함수는 xlsx을 불러와 데이터프레임으로 저장합니다. 

read_csv() 함수는 csv를 불러와 데이터프레임으로 저장합니다. 

data_update() 함수는 신규 데이터의 업데이트시 I 또는 U를 입력합니다. 

date_update() 함수는 신규 데이터의 업데이트시 날짜를 입력합니다. 

CheckDate() 함수는 인베스팅 닷컴의 일반 데이터 날짜를 표준화하는 함수 

CheckLength() 함수는 데이터의 크기를 확인하여 크기만큼 자르는 함수 

CheckVarchar() 함수는 재무 데이터 크기를 확인하여 클 경우 새로 삽입하는 함수 

CheckNumeric() 함수는 재무 데이터 숫자를 확인하는 함수 

-------------------------------------------------------------------



# Data Transform

In [3]:
info_path = "C:/Users/user/Desktop/유신영 업무폴더/BDC/1_BDC 데이터 정리/13_2023년 3분기/131_2023년 3분기 3차년도/2023년 3분기 3차년도 일반정보/적재전(가공전)/1차년도/"
fn_info_path = "C:/Users/user/Desktop/유신영 업무폴더/BDC/1_BDC 데이터 정리/13_2023년 3분기/131_2023년 3분기 3차년도/2023년 3분기 3차년도 일반정보/적재전(가공후)/1차년도/"
fmp_path = "C:/Users/user/Desktop/유신영 업무폴더/BDC/1_BDC 데이터 정리/13_2023년 3분기/131_2023년 3분기 3차년도/2023년 3분기 3차년도 재무정보/가공후/FMP/"
fn_fmp_path = "C:/Users/user/Desktop/유신영 업무폴더/BDC/1_BDC 데이터 정리/13_2023년 3분기/131_2023년 3분기 3차년도/2023년 3분기 3차년도 재무정보/적재전/FMP/"
dart_path = "C:/Users/user/Desktop/유신영 업무폴더/BDC/1_BDC 데이터 정리/13_2023년 3분기/131_2023년 3분기 3차년도/2023년 3분기 3차년도 재무정보/가공후/DART/"
fn_dart_path = "C:/Users/user/Desktop/유신영 업무폴더/BDC/1_BDC 데이터 정리/13_2023년 3분기/131_2023년 3분기 3차년도/2023년 3분기 3차년도 재무정보/적재전/FMP/"

In [4]:
ta = T.Analysis()

In [5]:
from tqdm import tqdm

In [27]:
for idx in tqdm(fmp_filepath):
    df = pd.read_excel(fmp_path + idx)
    df = df[df["accnn_yr"] >= 2022]
    df.to_excel(fmp_path + idx, index=False)

100%|██████████| 11/11 [00:41<00:00,  3.73s/it]


In [29]:
def info_transform(path, fn_path):
    for idx in tqdm(os.listdir(path)):
        t = T.Checker()
        t.read_excel(path + idx)
        limited_columns = t.df.columns[:58]
        t.df = t.df[limited_columns]
        try: t.df = ta.change_columns(t.df, info=True)
        except: print(idx)
        try: t.CheckDate()
        except: pass
        t.data_update(True)
        t.date_update()
        t.fndtn_dt()        
        t.df.to_excel(fn_path + idx, index=False)

In [30]:
def finan_transform(path, fn_path):
    for idx in tqdm(os.listdir(path)):
        try:
            t = T.Checker()
            t.read_excel(path + idx)
            t.df = ta.change_columns(t.df)
            t.CheckLength()
            t.CheckNumeric()
            t.CheckVarchar()
            try: t.CheckDate()
            except: pass
            t.data_update(True)
            t.date_update()
            t.fndtn_dt()
            t.df.to_excel(f"{fn_path}{idx}", index=False)
        except:
            t = T.Checker()
            t.read_excel(path + idx)
            t.CheckLength()
            t.CheckNumeric()
            t.CheckVarchar()
            t.data_update(True)
            t.date_update()
            t.fndtn_dt()
            t.df.to_excel(f"{fn_path}{idx}", index=False)

In [53]:
t = T.Checker()
idx = "네덜란드.xlsx"
t.read_excel(info_path + idx)
limited_columns = t.df.columns[:58]
t.df = t.df[limited_columns]
try: t.df = ta.change_columns(t.df, info=True)
except: print(idx)
try: t.CheckDate()
except: pass
t.data_update(True)
t.date_update()
t.fndtn_dt()        
t.df.to_excel(fn_info_path + idx, index=False)

c:\Users\user\Desktop\유신영 업무폴더\BDC\수정 후 BDC 개발모듈로 이동\Digital-Industry-Innovation\DataQuality\ETL\T.py:138: RuntimeWarning: invalid value encountered in cast
  if value.astype(int) > 21000000 or value.astype(int) < 18000000:


In [6]:
t = T.Checker()
t.read_excel(dart_path + os.listdir(dart_path)[0])

In [7]:
t.df = ta.change_columns(t.df, korea=True)

In [9]:
t.CheckLength()
t.CheckNumeric()
t.CheckVarchar()
try: t.CheckDate()
except: pass
t.data_update(True)
t.date_update()
t.fndtn_dt()

In [12]:
t.df.to_excel(fn_dart_path + os.listdir(dart_path)[0], index=False)

In [15]:
tt = EL.DataLoad()

In [16]:
cnt = 0
for idx in filepath:
    try:
        os.rename(path+idx, "fn/unload/" + tt.change_name(idx))
    except FileExistsError:
        cnt += 1
        os.rename(path + idx, "fn/unload/" + str(cnt) + tt.change_name(idx))

In [17]:
import pandas as pd
df = pd.read_excel("fn/fn_unfinished/Original_미국_2000.xlsx")

In [19]:
t.read_excel("fn/fn_unfinished/Original_미국_2000.xlsx")

In [25]:
for idx in tqdm(invest_fn):
    df = pd.read_excel(invest_fn_path + idx)
    df[df["accnn_yr"] >= 2022]
    df.to_excel(invest_fn_path + idx, index=False)

100%|██████████| 15/15 [03:27<00:00, 13.84s/it]


# Data Analysis

데이터베이스에서 데이터를 확인하는 클래스는 Analysis 입니다. 

read_excel() 함수는 xlsx을 불러와 데이터프레임으로 저장합니다. 

read_csv() 함수는 csv를 불러와 데이터프레임으로 저장합니다. 

Fail() 함수는 오류가 난 데이터를 데이터프레임에 넣기 위해 딕셔너리화 시키는 함수 

CheckDate_Duplicate() 함수는 날짜체크, 중복체크를 확인하는 함수 

CheckNumber() 함수는 전화번호가 유효한지 확인하는 함수 

In [1]:
import os
import sys
from tqdm import tqdm
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname("ETL"))))
from ETL import T

In [13]:
info_path = "C:/Users/user/Desktop/유신영 업무폴더/BDC/1_BDC 데이터 정리/13_2023년 3분기/131_2023년 3분기 3차년도/2023년 3분기 3차년도 일반정보/적재전(가공후)/1차년도/"
finan_path = "C:/Users/user/Desktop/유신영 업무폴더/BDC/1_BDC 데이터 정리/13_2023년 3분기/131_2023년 3분기 3차년도/2023년 3분기 3차년도 재무정보/적재전/INVESTING/"
filepath = os.listdir(finan_path)

In [14]:
ana = T.Analysis()

In [15]:
for idx in tqdm(filepath):
    ana.read_excel(finan_path + idx)
    ana.CheckDate_Duplicate(info=False)

  0%|          | 0/15 [00:00<?, ?it/s]


IndexError: list index out of range

In [7]:
ana.CheckDF.to_csv("2023-07-23_3분기 일반정보_데이터품질.csv", index=False)

In [6]:
ana.CheckDF.to_excel("오류율 검사.xlsx", index=False)

In [36]:
egnin_dict = {
    "일본": "tb_hb_jpn_egnin_m",
    "홍콩": "tb_hb_hkg_egnin_m",
    "말레이시아": "tb_hb_mys_egnin_m",
    "싱가포르": "tb_hb_sgp_egnin_m",
    "태국": "tb_hb_tha_egnin_m",
    "베트남": "tb_hb_vnm_egnin_m",
    "인도네시아": "tb_hb_idn_egnin_m",
    "인도": "tb_hb_ind_egnin_m",
    "미국": "tb_hb_usa_egnin_m",
    "캐나다": "tb_hb_can_egnin_m",
    "콜롬비아": "tb_hb_col_egnin_m",
    "멕시코": "tb_hb_mex_egnin_m",
    "네덜란드": "tb_hb_nld_egnin_m",
    "독일": "tb_hb_deu_egnin_m",
    "이탈리아": "tb_hb_ita_egnin_m",
    "프랑스": "tb_hb_fra_egnin_m",
    "영국": "tb_hb_gbr_egnin_m",
    "호주": "tb_hb_aus_egnin_m",
    "스위스": "tb_hb_che_egnin_m",
    "스페인": "tb_hb_esp_egnin_m"}
plcfi_dict = {
    "한국": "tb_hb_kor_plcfi_d",
    "베트남": "tb_hb_vnm_plcfi_d",
    "인도네시아": "tb_hb_idn_plcfi_d",
    "미국": "tb_hb_usa_plcfi_d",
    "일본": "tb_hb_jpn_plcfi_d",
    "홍콩": "tb_hb_hkg_plcfi_d",
    "말레이시아": "tb_hb_mys_plcfi_d",
    "싱가포르": "tb_hb_sgp_plcfi_d",
    "태국": "tb_hb_tha_plcfi_d",
    "인도": "tb_hb_ind_plcfi_d",
    "캐나다": "tb_hb_can_plcfi_d",
    "콜롬비아": "tb_hb_col_plcfi_d",
    "멕시코": "tb_hb_mex_plcfi_d",
    "네덜란드": "tb_hb_nld_plcfi_d",
    "독일": "tb_hb_deu_plcfi_d",
    "이탈리아": "tb_hb_ita_plcfi_d",
    "프랑스": "tb_hb_fra_plcfi_d",
    "영국": "tb_hb_gbr_plcfi_d",
    "호주": "tb_hb_aus_plcfi_d",
    "스위스": "tb_hb_che_plcfi_d",
    "스페인": "tb_hb_esp_plcfi_d"
}
plcfi_dict_en = {
    "korea": "tb_hb_kor_plcfi_d",
    "vietnam": "tb_hb_vnm_plcfi_d",
    "indonesia": "tb_hb_idn_plcfi_d",
    "usa": "tb_hb_usa_plcfi_d",
    "japan": "tb_hb_jpn_plcfi_d",
    "hongkong": "tb_hb_hkg_plcfi_d",
    "mylaysia": "tb_hb_mys_plcfi_d",
    "singapore": "tb_hb_sgp_plcfi_d",
    "thai": "tb_hb_tha_plcfi_d",
    "india": "tb_hb_ind_plcfi_d",
    "canada": "tb_hb_can_plcfi_d",
    "colombia": "tb_hb_col_plcfi_d",
    "mexico": "tb_hb_mex_plcfi_d",
    "netherland": "tb_hb_nld_plcfi_d",
    "germany": "tb_hb_deu_plcfi_d",
    "italy": "tb_hb_ita_plcfi_d",
    "france": "tb_hb_fra_plcfi_d",
    "uk": "tb_hb_gbr_plcfi_d",
    "australia": "tb_hb_aus_plcfi_d",
    "switzerland": "tb_hb_che_plcfi_d",
    "spain": "tb_hb_esp_plcfi_d"
}

In [58]:
def find_key_with_value(text, dictionary):
    for key, value in dictionary.items():
        if key in text:
            return value

In [66]:
def translate_file_name(path):
    for name in os.listdir(path):
        try:
            nm = name.split(".")[0]
            if "일반" in path: nm = egnin_dict[nm]
            elif "재무" in path: 
                try: nm = plcfi_dict[nm]
                except: nm = find_key_with_value(nm, plcfi_dict_en)
            os.rename(path + name, path + nm + ".xlsx")
        except:
            print(name)

In [67]:
plcfi_d = "C:/Users/user/Desktop/유신영 업무폴더/BDC/1_BDC 데이터 정리/13_2023년 3분기/131_2023년 3분기 3차년도/2023년 3분기 3차년도 재무정보/데이터베이스 적재/"

In [68]:
translate_file_name(plcfi_d)

finished_malaysia.xlsx
tb_hb_col_plcfi_d.xlsx
tb_hb_deu_plcfi_d.xlsx
tb_hb_fra_plcfi_d.xlsx
tb_hb_hkg_plcfi_d.xlsx
tb_hb_idn_plcfi_d.xlsx
tb_hb_ind_plcfi_d.xlsx
tb_hb_ita_plcfi_d.xlsx
tb_hb_jpn_plcfi_d.xlsx


In [36]:
translate_file_name(path)

KeyError: 'tb_hb_deu_egnin_m'

# FMP DATA MERGE

In [22]:
import pandas as pd
import os
import re

In [24]:
fmp_merge_path = "C:/Users/user/Desktop/유신영 업무폴더/BDC/1_BDC 데이터 정리/13_2023년 3분기/131_2023년 3분기 3차년도/2023년 3분기 3차년도 재무정보/적재전/FMP/"

In [35]:
temp = None
hangul = re.compile("[ㄱ-ㅎㅏ-ㅣ가-힣]+")
for idx in tqdm(os.listdir(fmp_merge_path)):
    nm = idx.split(".")[0]
    nm = str(hangul.findall(nm)[0])
    if temp == None:
        temp = nm
        df = pd.read_excel(fmp_merge_path + idx)
        print("start is " + idx)
        continue
    else:
        if temp == nm:
            append_df = pd.read_excel(fmp_merge_path + idx)
            df = df.append(append_df)
            print(idx + " merging...")
            continue
        else:
            df.to_excel(f"merged_{temp}.xlsx", index=False)
            temp = nm
            df = pd.read_excel(fmp_merge_path + idx)
            print("base is " + idx)
            continue
df.to_excel(f"merged_{str(temp)}.xlsx", index=False)

  9%|▉         | 1/11 [00:01<00:12,  1.26s/it]

start is 미국_10000.xlsx


 18%|█▊        | 2/11 [00:02<00:12,  1.35s/it]

미국_12000.xlsx merging...


 27%|██▋       | 3/11 [00:04<00:11,  1.40s/it]

...

영국_2642.xlsx merging...


 91%|█████████ | 10/11 [00:28<00:03,  3.33s/it]

base is 캐나다_1858.xlsx


100%|██████████| 11/11 [00:31<00:00,  2.88s/it]

base is 호주_2000.xlsx
